<a href="https://colab.research.google.com/github/avisekksarma/Fraud-detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import resample
import numpy as np
import lightgbm as lgb
%matplotlib inline

# disable the warning about settingwithcopy warning:
pd.set_option('chained_assignment',None)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import  GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold

In [ ]:
os.chdir('//content/gdrive/My Drive/Fraud_detection/')


In [ ]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [ ]:
train_df.head()

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V310,V314,V320,bank_type,full_addr,TransactionAmt_Log,Transaction_day_of_week,Transaction_hour,TransactionAmt_decimal,lastest_browser
0,2987000,0,86400,68.5,0.744522,13926,-999.0,150.0,0.011263,142.0,...,0.0,0.0,0.0,0.000469,0.039078,4.226834,0.0,0.0,500,0.0
1,2987001,0,86401,29.0,0.744522,2755,404.0,150.0,0.320414,102.0,...,0.0,0.0,0.0,0.043907,0.072388,3.367296,0.0,0.0,0,0.0
2,2987002,0,86469,59.0,0.744522,4663,490.0,150.0,0.651551,166.0,...,0.0,0.0,0.0,0.095658,0.044508,4.077537,0.0,0.0,0,0.0
3,2987003,0,86499,50.0,0.744522,18132,567.0,150.0,0.320414,117.0,...,354.0,0.0,0.0,0.043924,0.016046,3.912023,0.0,0.0,0,0.0
4,2987004,0,86506,50.0,0.055922,4497,514.0,150.0,0.320414,102.0,...,0.0,0.0,0.0,0.043907,0.006064,3.912023,0.0,0.0,0,0.0


In [ ]:
test_original = test_df.copy()
target = train_df['isFraud'].copy()
train_df = train_df.drop('isFraud', axis=1)
train_df.drop('TransactionDT', axis=1, inplace=True)
train_df.drop('TransactionID_x', axis=1, inplace=True)

test_df.drop('TransactionDT', axis=1, inplace=True)
test_df.drop('TransactionID_x', axis=1, inplace=True)

In [ ]:
print(test_original.shape)
print(train_df.shape)
print(test_df.shape)

(506691, 124)
(590540, 122)
(506691, 122)


In [ ]:
train_df.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,V310,V314,V320,bank_type,full_addr,TransactionAmt_Log,Transaction_day_of_week,Transaction_hour,TransactionAmt_decimal,lastest_browser
0,68.5,0.744522,13926,-999.0,150.0,0.011263,142.0,0.252288,315.0,87.0,...,0.0,0.0,0.0,0.000469,0.039078,4.226834,0.0,0.0,500,0.0
1,29.0,0.744522,2755,404.0,150.0,0.320414,102.0,0.252288,325.0,87.0,...,0.0,0.0,0.0,0.043907,0.072388,3.367296,0.0,0.0,0,0.0
2,59.0,0.744522,4663,490.0,150.0,0.651551,166.0,0.744976,330.0,87.0,...,0.0,0.0,0.0,0.095658,0.044508,4.077537,0.0,0.0,0,0.0
3,50.0,0.744522,18132,567.0,150.0,0.320414,117.0,0.744976,476.0,87.0,...,354.0,0.0,0.0,0.043924,0.016046,3.912023,0.0,0.0,0,0.0
4,50.0,0.055922,4497,514.0,150.0,0.320414,102.0,0.252288,420.0,87.0,...,0.0,0.0,0.0,0.043907,0.006064,3.912023,0.0,0.0,0,0.0


In [ ]:
test_df.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,V310,V314,V320,bank_type,full_addr,TransactionAmt_Log,Transaction_day_of_week,Transaction_hour,TransactionAmt_decimal,lastest_browser
0,31.95,0.71244,10409,111.0,150.0,0.660920,226.0,0.759873,170.0,87.0,...,47.950001,0.000000,0.0,0.485173,0.003570,3.464172,2.0,0.0,949,1.0
1,49.00,0.71244,4272,111.0,150.0,0.660920,226.0,0.759873,299.0,87.0,...,280.000000,0.000000,0.0,0.485173,0.076368,3.891820,2.0,0.0,0,1.0
2,171.00,0.71244,4476,574.0,150.0,0.660920,226.0,0.759873,472.0,87.0,...,1058.000000,0.000000,263.0,0.485173,0.014261,5.141664,2.0,0.0,0,0.0
3,284.95,0.71244,10989,360.0,150.0,0.660920,166.0,0.759873,205.0,87.0,...,0.000000,282.540009,0.0,0.088421,0.010460,5.652314,2.0,0.0,949,1.0
4,67.95,0.71244,18018,452.0,150.0,0.312161,117.0,0.759873,264.0,87.0,...,183.850006,183.850006,0.0,0.041682,0.064556,4.218772,2.0,0.0,950,1.0


In [ ]:
target.shape

(590540,)

In [ ]:
SEED = 10

params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'max_bin': 510,
          'subsample_for_bin': 200,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }


mdl = lgb.LGBMClassifier(
          boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = 5,
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'])

# To view the default model parameters:
mdl.get_params().keys()

dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'subsample', 'subsample_for_bin', 'subsample_freq', 'silent', 'max_bin'])

In [ ]:
# # # Grid Search

# gridParams = {
#     'learning_rate': [0.005, 0.01],
#     'n_estimators': [8,16,24],
#     'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
#     'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
#     'objective' : ['binary'],
#     'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
#     'random_state' : [500],
#     'colsample_bytree' : [0.64, 0.65, 0.66],
#     'subsample' : [0.7,0.75],
#     'reg_alpha' : [1,1.2],
#     'reg_lambda' : [1,1.2,1.4],
#     }


# grid = GridSearchCV(mdl, gridParams, verbose=1, cv=2, n_jobs=-1)
# # Run the grid
# grid.fit(train_df, target)

# # Print the best parameters found
# print(grid.best_params_)
# print(grid.best_score_)


In [ ]:
# # Use the best model

# params['colsample_bytree'] = grid.best_params_['colsample_bytree']
# params['learning_rate'] = grid.best_params_['learning_rate']
# params['max_bin'] = grid.best_params_['max_bin']
# params['num_leaves'] = grid.best_params_['num_leaves']
# params['reg_alpha'] = grid.best_params_['reg_alpha']
# params['reg_lambda'] = grid.best_params_['reg_lambda']
# params['subsample'] = grid.best_params_['subsample']


In [ ]:
splits = 6
#Use Group K fold Split by months
# folds = StratifiedKFold(n_splits=splits, shuffle=True, random_state=123)
folds = KFold(n_splits = splits)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

In [ ]:
# plot Precision Recall Curve
def plot_pre_curve(y_valid_df,pred):
    precision, recall, thresholds = precision_recall_curve(y_valid_df, pred)
    plt.plot([0, 1], [0.5, 0.5], linestyle='--')
    # plot the precision-recall curve for the model
    plt.plot(recall, precision, marker='.')
    plt.title("precision recall curve")
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the plot
    plt.show()

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    x_train_df, y_train_df = train_df.iloc[trn_idx], target.iloc[trn_idx]
    x_valid_df, y_valid_df = train_df.iloc[val_idx], target.iloc[val_idx]

    trn_data = lgb.Dataset(x_train_df, label=y_train_df)
    val_data = lgb.Dataset(x_valid_df, label=y_valid_df)

    clf = lgb.train(params,
                    trn_data,
                    10000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=500)

    pred = clf.predict(x_valid_df)
    oof[val_idx] = pred
    print( "  auc = ", roc_auc_score(y_valid_df, pred) )
    # lets see Precision_Recall curve
    plot_pre_curve(y_valid_df,pred)
    predictions += clf.predict(test_df) / splits

Fold 0


TypeError: ignored

In [ ]:
# # sorted(zip(clf.feature_importances_, X.columns), reverse=True)
# feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(importance_type='split',iteration=None),train_df.columns)), columns=['Value','Feature'])

# plt.figure(figsize=(15, 20))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.show()
# plt.savefig('lgbm_importances-01.png')

In [ ]:
#tree visualization
graph = lgb.create_tree_digraph(clf, tree_index=3, name='Tree3' )
graph.graph_attr.update(size="110,110")
graph

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')


In [ ]:
sample_submission['isFraud'] = clf.predict(test_df)

In [ ]:
sample_submission.head(20)

In [ ]:
a=sample_submission['isFraud']
ones_count=0
zeros_count =0
for i in a:
  if i>0.5:
    ones_count = ones_count+1
  else :
    zeros_count = zeros_count + 1

print(ones_count , zeros_count)
total = ones_count + zeros_count
print("Ones {}".format(ones_count * 100 / total))
print("zeros {}".format(zeros_count * 100 / total))

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.shape

In [ ]:
# sample_submission = pd.read_csv('sample_submission.csv')
# sample_submission.isFraud = predictions
sample_submission.to_csv('sub_lgb.csv',index=True)


In [ ]:
predictions.shape

In [ ]:
sample_submission.shape